## Capstone Project 
#### Data source:
https://www.consumerfinance.gov/data-research/consumer-complaints/

#### Data Description:
This data set comes from CFPB and complaints that the CFPB sends to companies for response are published in the Consumer Complaint Database after the company responds, confirming a commercial relationship with the consumer, or after 15 days, whichever comes first.


#### How we access this data:
There are two ways, by downloading the dataset or through API.
In our analysis, we will just be downloading the full set of dataset that is from the period `2011-12-01` to `2022-10-01`.



In [ ]:
# If you are running this notebook in Jupyter lab, please make sure you run this part so that the visualizations will show up.
# !pip install "jupyterlab>=3" "ipywidgets>=7.6"
# !pip install plotly==5.10.0
# !pip install altair

In [ ]:
import pandas as pd
import sklearn
#import altair as alt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
#alt.data_transformers.disable_max_rows()

In [ ]:
import zipfile
# with zipfile.ZipFile('complaints-2022-10-23_09_54.csv.zip') as zip:
#     with zip.open('complaints-2022-10-23_09_54.csv') as myZip:
#         df = pd.read_csv(myZip) 
df=pd.read_csv('complaints-2022-10-23_09_54.csv')

In [ ]:
df.shape

#### Let's see what columns we have, and its description

| Column Name | Description | Data Type | Notes |
| --- | --- | ---- | --- |
| Date received | The date the CFPB received the complaint | date & time |  |
| Product | The type of product the consumer identified in the complaint | plain text| This field is a categorical variable. |
| Sub-product | The type of sub-product the consumer identified in the complaint | plain text| This field is a categorical variable. Not all Products have Sub-products. |
| Issue | The issue the consumer identified in the complaint | plain text| This field is a categorical variable. Possible values are dependent on Product. |
| Sub-issue | The sub-issue the consumer identified in the complaint | plain text | Possible values are dependent on product and issue. Not all Issues have corresponding Sub-issues. |
| Consumer complaint narrative | Consumer complaint narrative is the consumer-submitted description of "what happened" from the complaint. Consumers must opt-in to share their narrative. We will not publish the narrative unless the consumer consents, and consumers can opt-out at any time. The CFPB takes reasonable steps to scrub personal information from each complaint that could be used to identify the consumer. | plain text | Consumers' descriptions of what happened are included if consumers consent to publishing the description and after we take steps to remove personal information.  |
| Company public response | The company's optional, public-facing response to a consumer's complaint. Companies can choose to select a response from a pre-set list of options that will be posted on the public database. For example, "Company believes complaint is the result of an isolated error." | plain text | Companies' public-facing responses to complaints are included if companies choose to publish one. Companies may select a public response from a set list of options as soon as they respond to the complaint, but no later than 180 days after the complaint was sent to the company for response. |
| Company | The complaint is about this company. | plain text | This field is a categorical variable. |
| State | The state of the mailing address provided by the consumer | plain text | This field is a categorical variable. |
| ZIP code | The mailing ZIP code provided by the consumer. | plain text | Mailing ZIP code provided by the consumer. This field may: i) include the first five digits of a ZIP code; ii) include the first three digits of a ZIP code (if the consumer consented to publication of their complaint narrative); or iii) be blank (if ZIP codes have been submitted with non-numeric values, if there are less than 20,000 people in a given ZIP code, or if the complaint has an address outside of the United States). |
| Tags | Data that supports easier searching and sorting of complaints submitted by or on behalf of consumers. For example, complaints where the submitter reports the age of the consumer as 62 years or older are tagged, ‘Older American.’ Complaints submitted by or on behalf of a servicemember or the spouse or dependent of a servicemember are tagged, ‘Servicemember.’ Servicemember includes anyone who is active duty, National Guard, or Reservist, as well as anyone who previously served and is a Veteran or retiree.  | plain text |   |
| Consumer consent provided? | Identifies whether the consumer opted in to publish their complaint narrative. We do not publish the narrative unless the consumer consents and consumers can opt-out at any time. | plaint text | This field shows whether a consumer provided consent to publish their complaint narrative. |
| Submitted via | How the complaint was submitted to the CFPB | plain text | This field is a categorical variable. |
| Date sent to company | The date the CFPB sent the complaint to the company | date & time |  |
| Company response to consumer | This is how the company responded. For example, "Closed with explanation." | plain text | This field is a categorical variable. |
| Timely response? | Whether the company gave a timely response | plain text | yes/no |
| Consumer disputed? | Whether the consumer disputed the company’s response | plain text | yes/no <br> N/A: The Bureau discontinued the consumer dispute option on April 24, 2017. |
| Complaint ID | The unique identification number for a complaint | number |   |

In [ ]:
df.columns.to_list()

In [ ]:
### Find missing values
df.isnull().sum()

In [ ]:
df['Date_received_dt'] = pd.to_datetime(df['Date received'], format='%m/%d/%y')
df['Date_sent_to_company_dt'] = pd.to_datetime(df['Date sent to company'], format='%m/%d/%y')
df['time_diff'] = df['Date_sent_to_company_dt']-df['Date_received_dt']
# The percentage of complaints that don't sent to the company on the same date.
len(df[df['time_diff']!='0 days'])/len(df)*100

In [ ]:
# Let's take a quick look at the categorical variables
categorical_var=['Product','Sub-product',
'Issue',
'Sub-issue']

for i in categorical_var:
    print(df[i].unique())

In [ ]:
# alt.Chart(df).mark_bar().encode(x=alt.X('yearmonth(Date_received_dt):O', axis=alt.Axis(labelAngle=325)),
#                     y=alt.Y('count(distinct Complaint ID):Q', scale=alt.Scale(zero=False)),)

# text = bars.mark_text(
#     align='left',
#     baseline='middle',
#     dy=-5  # Nudges text to right so it doesn't appear on top of the bar
# ).encode(
#     text='count(distinct Complaint ID):Q'
# )

# (bars + text).properties(height=300)

In [ ]:
fig = px.histogram(df, x="Date_received_dt")
fig.show()

In [ ]:
#Number of cases by different product. --- Looks like that Product of Credit reporting, credit repair services remain as the top 1 product that receives complaints
fig = px.histogram(df, x="Date_received_dt", color='Product')
fig.show()

In [ ]:
P = df.groupby(['Date_received_dt','Timely response?'])['Complaint ID'].count().reset_index()
P

In [ ]:
P['Percentage'] = P.groupby('Date_received_dt')['Complaint ID'].apply(lambda x: x*100/x.sum())

In [ ]:
P_yes=P[P['Timely response?']=='Yes']

In [ ]:
#Number of cases by different Timely Response. 
#--- Looks like companies are always consistent throughout the year, except that we see a decrease around Nov 2017.

fig = px.line(P_yes, x='Date_received_dt',y="Percentage")
fig.show()

In [ ]:
#Here's the top 10 Company by number of complaints
top10=df.groupby('Company')['Complaint ID'].count().reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)
top10_company=top10.Company.to_list()
top10_df=df[df['Company'].isin(top10_company)]

In [ ]:
#Group by the data to year and month level
P2=top10_df.groupby([top10_df['Date_received_dt'].dt.to_period('M').rename('Year-Month'),
                     'Company','Timely response?'])['Complaint ID'].count().reset_index(name='count')

P2['Percentage']=P2.groupby(['Year-Month'
           ,'Company'])['count'].apply(lambda x: x*100/x.sum())



In [ ]:
#Keep only timely response percentage.
P2_yes=P2[P2['Timely response?']=='Yes']

In [ ]:
#By looking at this chart, we could see that the timely reponse rate dropped for top 10 financial services. 
# For wells-fargo, we see dramastic drop to a rate below 70 around Oct 2016 and Jan 2017, and dropped below 90 around Oct 2019 and Dec 2019.
# For Bank of America, we see drop below 60 around Jun 2021.
# For Equifax, we see drop below 80 around May 2017, and below 70 around Nov 2017.
# The other financial services has a stable response rate over time.
fig = px.line(P2_yes, x=P2_yes['Year-Month'].dt.to_timestamp(),y="Percentage",color='Company')
fig.show()

In [ ]:
P3=top10_df[top10_df['time_diff']!='0 days'].groupby(['Company','time_diff'])['Complaint ID'].count().reset_index(name='count')
P3['Percentage']=P3.groupby(['Company'])['count'].apply(lambda x: x*100/x.sum())


In [ ]:
for i in list(P3.Company.unique()):
    Company_df=P3[P3['Company']==i]
    Company_df['time_diff_str']=Company_df['time_diff'].astype(str)
    fig=px.bar(Company_df,x='time_diff_str',y='Percentage',title=f'Company:{i}')
    fig.show() 

In [ ]:
P4=df[df['time_diff']!='0 days'].groupby(['Product','time_diff'])['Complaint ID'].count().reset_index(name='count')
P4['Percentage']=P4.groupby(['Product'])['count'].apply(lambda x: x*100/x.sum())

In [ ]:
for i in list(P4.Product.unique()):
    Product_df=P4[P4['Product']==i]
    Product_df['time_diff_str']=Product_df['time_diff'].astype(str)
    fig=px.bar(Product_df,x='time_diff_str',y='Percentage',title=f'Product:{i}')
    fig.show()  